In [196]:
from google.cloud import vision
import pandas as pd

In [197]:
import os
import glob
import base64

In [ ]:
import io
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types
import json
from google.protobuf.json_format import MessageToJson




from fnmatch import fnmatch


client = vision.ImageAnnotatorClient()
image = vision.types.Image()

root = '/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/'
pattern = "*.png"

cols = ['AdId','num_faces','face_joylikelihood', 'face_sorrowLikelihood', 'face_angerLikelihood','face_surpriseLikelihood','face_description_confidence',
                    'landmark_features','labels_description','labels_descriptions_confidence','logo_description','logo_description_confidence','text_descriptions',
                   'object_descriptions','object_descriptions_confidence','isAdult','isSpoof','isMedical','isViolence','isRacy']
df =  pd.DataFrame(columns = cols)  



for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            #print os.path.join(path, name)
            print (path + "/" + name)

            #client = vision.ImageAnnotatorClient()
            #image_path = '/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/13.png'

            # Loads the image into memory
            filename = (path  + "/" + name)
            #filename =  '/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/13.png'
            with io.open(filename, 'rb') as image_file:
                content = image_file.read()

            image = types.Image(content=content)


            query = {

                "image": image ,

                  "features": [
                    {
                      "type": "LANDMARK_DETECTION"
                    },
                    {
                      "type": "FACE_DETECTION"
                    },
                    {
                      "type": "OBJECT_LOCALIZATION"
                    },
                    {
                      "type": "LOGO_DETECTION"
                    },
                    {
                      "type": "LABEL_DETECTION"
                    },
                    {
                      "type": "DOCUMENT_TEXT_DETECTION"
                    },
                    {
                      "type": "SAFE_SEARCH_DETECTION"
                    },
                    {
                      "type": "IMAGE_PROPERTIES"
                    },
                    {
                      "type": "CROP_HINTS"
                    },
                    {
                      "type": "WEB_DETECTION"
                    }
                  ],

                }



            response = client.annotate_image(query)
            #print(type(response))
            
            #response_json = MessageToJson(response)
            #print(type(response_json))
            
            # FACE ANNOTATIONS
            joylikelihood = []
            sorrowLikelihood = []
            angerLikelihood = []
            surpriseLikelihood = []
            face_description_score = []
            num_faces = len(response.face_annotations)
            for i in range(len(response.face_annotations)):
                joylikelihood.append(vision.enums.Likelihood(response.face_annotations[i].joy_likelihood).name)
                sorrowLikelihood.append(vision.enums.Likelihood(response.face_annotations[i].sorrow_likelihood).name)
                angerLikelihood.append(vision.enums.Likelihood(response.face_annotations[i].anger_likelihood).name)
                surpriseLikelihood.append(vision.enums.Likelihood(response.face_annotations[i].surprise_likelihood).name)
                face_description_score.append(response.face_annotations[i].detection_confidence)
            
            #Landmark Features
            #response.landmark_annotations
            landmarkFeatures = []
            for i in range(len(response.landmark_annotations)):
                landmarkFeatures.append(str(response.landmark_annotations[i].description).encode('utf-8'))
                
                
            
            #Label Features
            labels_description = []
            labels_scores = []
            for i in range(len(response.label_annotations)):
                labels_description.append(str(response.label_annotations[i].description).encode('utf-8'))
                labels_scores.append(response.label_annotations[i].score)
                #print(type(response.label_annotations[i].description.encode('utf-8')))
                
            #Logo Descriptions
            logo_description = []
            logo_description_score = []
            for i in range(len(response.logo_annotations)):
                logo_description.append(str(response.logo_annotations[i].description.encode('utf-8')))
                logo_description_score.append(response.logo_annotations[i].score)
                
            #TEXT Features    
            text_descriptions = []
            for i in range(len(response.text_annotations)):
                if(len(response.text_annotations[i].description) > 2):
                    text_descriptions.append(str(response.text_annotations[i].description.encode('utf-8')))
                
            #Safe search
            isAdult = vision.enums.Likelihood(response.safe_search_annotation.adult).name
            isSpoof = vision.enums.Likelihood(response.safe_search_annotation.spoof).name
            isMedical= vision.enums.Likelihood(response.safe_search_annotation.medical).name
            isViolence = vision.enums.Likelihood(response.safe_search_annotation.violence).name
            isRacy = vision.enums.Likelihood(response.safe_search_annotation.racy).name
            
            #Object description
            objects_descriptions = []
            object_score = []
            for i in range(len(response.localized_object_annotations)):
                objects_descriptions.append(str(response.localized_object_annotations[i].name))
                object_score.append(response.localized_object_annotations[i].score)
            
            
            ADId = "A" + str(path.split(os.sep)[-1]).zfill(2) + "_" + str(os.path.splitext(name)[0]).zfill(2)
            #print (path.split(os.sep))
            print (ADId)
            
            df = df.append({'AdId' : ADId ,
                            'num_faces' : num_faces,
                            'face_joylikelihood': joylikelihood , 
                            'face_sorrowLikelihood' : sorrowLikelihood , 
                            'face_angerLikelihood' : angerLikelihood ,
                            'face_surpriseLikelihood' : surpriseLikelihood ,
                            'face_description_confidence': face_description_score,
                            'landmark_features' : landmarkFeatures,
                            'labels_description' : labels_description,
                            'labels_descriptions_confidence' : labels_scores , 
                            'logo_description' : logo_description , 
                            'logo_description_confidence' : logo_description_score , 
                            'text_descriptions' : text_descriptions,
                            'object_descriptions' : objects_descriptions,
                            'object_descriptions_confidence': object_score,
                            'isAdult' : isAdult , 
                            'isSpoof':isSpoof,'isMedical':isMedical,'isViolence':isViolence,'isRacy':isRacy
                           }, ignore_index=True)

/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/8.png
A20_08
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/9.png
A20_09
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/14.png
A20_14
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/15.png
A20_15
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/12.png
A20_12
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/13.png
A20_13
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/11.png
A20_11
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/10.png
A20_10
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/4.png
A20_04
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/20/5.png
A

A16_11
/Users/praveenkrishna/Documents/AI hakathon/ads16-dataset/ADS16_Benchmark_part1/Ads/16/10.png


In [ ]:
    #response
    df
    

In [ ]:
    df.to_csv('Ads_images_annotated.csv',index = False, header=True)

# TF Data Validate 

In [3]:
import tensorflow_data_validation as tfdv

/Users/praveenkrishna/Library/Python/2.7/lib/python/site-packages/apache_beam/__init__.py:84: UserWarning: You are using Apache Beam with Python 2. New releases of Apache Beam will soon support Python 3 only.
  'You are using Apache Beam with Python 2. '


In [47]:
df1 = pd.read_csv('Ads_images_annotated.csv')

In [48]:
df1.head()

,AdId,num_faces,face_joylikelihood,face_sorrowLikelihood,face_angerLikelihood,face_surpriseLikelihood,face_description_confidence,landmark_features,labels_description,labels_descriptions_confidence,...,text_descriptions,object_descriptions,object_descriptions_confidence,isAdult,isSpoof,isMedical,isViolence,isRacy,Joined,temp
0,A00_08,1,['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],[0.4056199789047241],[],"['Product', 'Shoulder', 'Bag', 'Joint', 'Neck'...","[0.9529739618301392, 0.9508122801780701, 0.864...",...,['Simplified Style Tote Elegant\nShoulder Bag ...,"['Person', 'Bag']","[0.8898026347160339, 0.8705868721008301]",UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY,POSSIBLE,"Product,Shoulder,Bag,Joint,Neck,Beige,Waist,Co...",['Product']
1,A00_09,0,[],[],[],[],[],[],"['Glasses', 'Footwear', 'Eyewear', 'Font', 'Br...","[0.9058366417884827, 0.7913055419921875, 0.738...",...,['Rivacci\nSee Colour Options\nSunglasses - Vi...,"['Sunglasses', 'Bag', 'Glasses']","[0.9405239224433899, 0.6796632409095764, 0.573...",VERY_UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY,UNLIKELY,"Glasses,Footwear,Eyewear,Font,Brand,Leather,Logo",['Font']
2,A00_14,1,['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],[0.9646230340003967],[],"['Text', 'Arm', 'Joint', 'Shoulder', 'Font', '...","[0.9121394753456116, 0.8889164328575134, 0.778...",...,['American Apparel\nContinue to\nShopping Site...,"['Person', 'Top', 'Person']","[0.8460196852684021, 0.8092377185821533, 0.786...",UNLIKELY,LIKELY,UNLIKELY,UNLIKELY,VERY_LIKELY,"Text,Arm,Joint,Shoulder,Font,Neck,Leg,Brassier...","['Text', 'Font']"
3,A00_15,2,"['VERY_LIKELY', 'VERY_UNLIKELY']","['VERY_UNLIKELY', 'VERY_UNLIKELY']","['VERY_UNLIKELY', 'VERY_UNLIKELY']","['VERY_UNLIKELY', 'VERY_UNLIKELY']","[0.8551092743873596, 0.8189187049865723]",[],"['Product', 'Font', 'Leisure', 'Recreation', '...","[0.934798538684845, 0.6857237815856934, 0.6809...",...,['Sandals\nLuxury Included Vacations\nSAVE\nUP...,"['Person', 'Person', 'Packaged goods', 'Person']","[0.8373076319694519, 0.8066036105155945, 0.769...",UNLIKELY,VERY_UNLIKELY,POSSIBLE,VERY_UNLIKELY,POSSIBLE,"Product,Font,Leisure,Recreation,Skin care,Vaca...","['Product', 'Font']"
4,A00_16,0,[],[],[],[],[],[],"['Product', 'Text', 'Material property', 'Font...","[0.9413959383964539, 0.8811547756195068, 0.699...",...,"[""WALLETS\nTTT WALLETS\nThe perfect home for y...","['Packaged goods', 'Packaged goods', 'Packaged...","[0.8149046301841736, 0.8114873170852661, 0.771...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,"Product,Text,Material property,Font,Writing im...","['Product', 'Text', 'Font']"


In [9]:
BASE_DIR = os.getcwd()
dfpath = os.path.join(BASE_DIR, 'Ads_images_annotated.csv')

ad_image_stats = tfdv.generate_statistics_from_csv(dfpath)

In [10]:
tfdv.visualize_statistics(ad_image_stats)

# Encoding

In [168]:
df1 = pd.read_csv('Ads_images_annotated.csv')

In [169]:
df1.head()

,AdId,num_faces,face_joylikelihood,face_sorrowLikelihood,face_angerLikelihood,face_surpriseLikelihood,face_description_confidence,landmark_features,labels_description,labels_descriptions_confidence,logo_description,logo_description_confidence,text_descriptions,object_descriptions,object_descriptions_confidence,isAdult,isSpoof,isMedical,isViolence,isRacy
0,A20_08,0,[],[],[],[],[],[],"['Text', 'Font', 'Logo', 'Brand']","[0.9540841579437256, 0.8314544558525085, 0.793...",[],[],"['iviy\nTM\nMySocialCircle.net - MEET\nPEOPLE,...",[],[],VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
1,A20_09,2,"['VERY_UNLIKELY', 'VERY_UNLIKELY']","['VERY_UNLIKELY', 'VERY_UNLIKELY']","['VERY_UNLIKELY', 'VERY_UNLIKELY']","['VERY_UNLIKELY', 'VERY_UNLIKELY']","[0.9458692073822021, 0.8170527815818787]",[],"['Text', 'Font', 'Photography']","[0.8692919611930847, 0.6857237815856934, 0.624...",[],[],['Online Dating Tips and Advices 17\nJul 2013\...,"['Person', 'Person', 'Person', 'Clothing', 'Cl...","[0.7279245257377625, 0.6595233082771301, 0.655...",VERY_UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY
2,A20_14,1,['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],[0.8977615833282471],[],"['Product', 'Mobile phone', 'Gadget', 'Smartph...","[0.9553973078727722, 0.9504517316818237, 0.948...",[],[],"[""tinder\ntinder\nDiscover those around you.\n...","['Mobile phone', 'Mobile phone', 'Person']","[0.8979392051696777, 0.8948755264282227, 0.862...",UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,UNLIKELY
3,A20_15,1,['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],[0.9697074890136719],[],"['Product', 'Text', 'Font', 'Line', 'Magenta',...","[0.9462587833404541, 0.9324976801872253, 0.804...",[],[],['LOVOO @lovoo\n13 Jun\n12 Millionen Nutzer. C...,"['Packaged goods', 'Person']","[0.708368718624115, 0.5171643495559692]",VERY_UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY,UNLIKELY
4,A20_12,1,['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],['VERY_UNLIKELY'],[0.956528902053833],[],"['Skin', 'Beauty', 'Text', 'Font', 'Dress', 'H...","[0.9137958288192749, 0.882392942905426, 0.8811...","['Bebe Stores', 'Sur La Table']","[0.6830872297286987, 0.6693288683891296]",['badoo\nMEET NEW PEOPLE.\n150 MILLION OF THEM...,"['Person', 'Hat', 'Dress']","[0.9053522944450378, 0.8445079922676086, 0.667...",VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,POSSIBLE


In [ ]:
encoded_df = df[['AdId']]

## Label Description

In [ ]:
#df['Joined'] = df.labels_description.str.replace('\[|\]|\'', '').astype(str)
df['Joined'] = df.labels_description.apply(lambda x: ','.join(map(str, x)))

In [ ]:
top_25_labels = df.Joined.apply(lambda x: pd.value_counts(x.split(","))).sum(axis = 0).sort_values(ascending=False).nlargest(n=25, keep='first').index.values.tolist()


In [ ]:
df['topK_labels_description'] = df['labels_description'].apply(lambda x: [item for item in x if item in top_25_labels])


In [ ]:
df[['topK_labels_description' ,'Joined','labels_description']]

In [ ]:
temp_encoded_df = df['topK_labels_description'].str.join('|').str.get_dummies()
temp_encoded_df = temp_encoded_df.add_prefix('ad_is')
temp_encoded_df

In [ ]:
encoded_df = pd.merge(encoded_df, encoded_temp_df, left_index=True, right_index=True)
encoded_df

## Face

In [ ]:
encoded_df['ad_num_faces'] = df['num_faces']
#encoded_df['AdId'] = df['AdId']
encoded_df

## isAdult 	isSpoof 	isMedical 	isViolence 	isRacy

In [ ]:
temp_df = df[["isAdult" , "isSpoof" , "isMedical" , "isViolence" , "isRacy" ]]

In [ ]:
#temp_df["isAdult"] = temp_df["isAdult"].replace({"UNLIKELY": "UNLIKELY","VERY_UNLIKELY": "UNLIKELY", "POSSIBLE": "LIKELY","VERY_LIKELY":"LIKELY","LIKELY":"LIKELY"})
#temp_df["isSpoof"] = temp_df["isSpoof"].replace({"UNLIKELY": "UNLIKELY","VERY_UNLIKELY": "UNLIKELY", "POSSIBLE": "LIKELY","VERY_LIKELY":"LIKELY","LIKELY":"LIKELY"})
#temp_df["isMedical"] = temp_df["isMedical"].replace({"UNLIKELY": "UNLIKELY","VERY_UNLIKELY": "UNLIKELY", "POSSIBLE": "LIKELY","VERY_LIKELY":"LIKELY","LIKELY":"LIKELY"})
#temp_df["isViolence"] = temp_df["isViolence"].replace({"UNLIKELY": "UNLIKELY","VERY_UNLIKELY": "UNLIKELY", "POSSIBLE": "LIKELY" ,"VERY_LIKELY":"LIKELY","LIKELY":"LIKELY"})
#temp_df["isRacy"] = temp_df["isRacy"].replace({"UNLIKELY": "UNLIKELY","VERY_UNLIKELY": "UNLIKELY", "POSSIBLE": "LIKELY" , "VERY_LIKELY":"LIKELY","LIKELY":"LIKELY"})

In [ ]:
#temp_df

In [ ]:
temp_encoded_df = pd.get_dummies(temp_df, prefix=["ad_isAdult" , "ad_isSpoof" , "ad_isMedical" , "ad_isViolence" , "ad_isRacy"],drop_first=True)

In [ ]:
encoded_df = pd.merge(encoded_df, temp_encoded_df, left_index=True, right_index=True)

In [ ]:
encoded_df

# object description

In [ ]:
#object_descriptions
df['Joined'] = df.object_descriptions.apply(lambda x: ','.join(map(str, x)))

In [ ]:
top_25_objects = df.Joined.apply(lambda x: pd.value_counts(x.split(","))).sum(axis = 0).sort_values(ascending=False).nlargest(n=25, keep='first')#.index.values.tolist()
#top_25_objects

In [ ]:
df['topK_objects_description'] = df['object_descriptions'].apply(lambda x: [item for item in x if item in top_25_objects])


In [ ]:
temp_encoded_df = df['topK_objects_description'].str.join('|').str.get_dummies()
temp_encoded_df = temp_encoded_df.add_prefix('ad_is')

In [ ]:
temp_encoded_df

In [ ]:
encoded_df = pd.merge(encoded_df, temp_encoded_df, left_index=True, right_index=True)

In [ ]:
encoded_df

In [ ]:
encoded_df.to_csv('Ads_images_annotated_encoded.csv',index = False, header=True)

# Explore 

## Face Features

In [30]:
    #response.face_annotations[0]
    

IndexError: list index (0) out of range

In [33]:
    #response.face_annotations[0].joy_likelihood
    #print((response.face_annotations))
    joylikelihood = []
    sorrowLikelihood = []
    angerLikelihood = []
    surpriseLikelihood = []
    for i in range(len(response.face_annotations)):
        joylikelihood.append(response.face_annotations[i].joy_likelihood)
        sorrowLikelihood.append(response.face_annotations[i].sorrow_likelihood)
        angerLikelihood.append(response.face_annotations[i].anger_likelihood)
        surpriseLikelihood.append(response.face_annotations[i].surprise_likelihood)


[]


## Landmark Features

In [40]:
    response.landmark_annotations
    landmarkFeatures = []
    for i in range(len(response.landmark_annotations)):
        landmarkFeatures.append(response.landmark_annotations[i].description)
    

In [35]:
    response.landmark_annotations

[]

## Label Features

In [13]:
    labels_description = []
    labels_scores = []
    for i in range(len(response.label_annotations)):
        labels_description.append(response.label_annotations[i].description)
        labels_scores.append(response.label_annotations[i].score)
        
    print(labels_scores)

[0.9708330035209656, 0.9509816765785217, 0.6661911010742188, 0.5848097205162048, 0.5733055472373962, 0.5518041253089905]


## Text Annotation

In [8]:
    response.text_annotations[0].description
    text_descriptions = []
    for i in range(len(response.text_annotations)):
        text_descriptions.append(response.text_annotations[i].description)
        
    (str(text_descriptions[0]))
    

'Blue Shoes - 25% Off\nFree Shipping, Exchanges & Returns\nBlue Shoes at anesshoes.com! Code: BLUE\nwww.anesshoes.com/colors\n'

## Safe search


In [123]:
    response.safe_search_annotation

adult: VERY_UNLIKELY
spoof: VERY_UNLIKELY
medical: VERY_UNLIKELY
violence: VERY_UNLIKELY
racy: VERY_UNLIKELY

In [121]:
    
    vision.enums.Likelihood(response.safe_search_annotation.adult).name
    vision.enums.Likelihood(response.safe_search_annotation.spoof).name
    vision.enums.Likelihood(response.safe_search_annotation.medical).name
    vision.enums.Likelihood(response.safe_search_annotation.violence).name
    vision.enums.Likelihood(response.safe_search_annotation.racy).name
    
    
    

'VERY_UNLIKELY'

## Image properties

In [21]:
    #response.dominant_colors_annotation